# XPN 2020 One Vote Wonders

The main play list for [XPN 202](https://xpn.org/music-artist/885-countdown/2020/),
was paused just after mindnight to allow the last 100 to be played starting at 8am.
From 6 to 8, there is a mini playlist of One Vote Wonders,
songs that got only one vote.
Let's see if we can throw something together for that in real time.
(oversleeping didn't help)

In [1]:
%matplotlib inline
from IPython.display import display, HTML

# Status

Well just kind of starting and it already over.
We'll see if I can think of something interestin.

## Loading the Data

Most people are probably not too interested in how I pull the data, clean it and augment it.
If you do the details are in my [Data Loading notebook](DataLoading.ipynb).
If you want a copy of the raw data to play with yourself, 
feel free to look at [xpn2020_onsies.csv](./data/xpn2020_onsies.csv).
If you do something interesting with the data,
please let me know.
And tell the station and post to [twitter](https://twitter.com) with the hash tag `#XPN2020`.
I'm sure other listeners will be glad to see what you have done.

In [2]:
import pandas as pd
from datetime import date, datetime, time, timedelta
from os import path
data_dir = './data'
playlist_file = path.join(data_dir, 'xpn2020_onsies.csv')
playlist = pd.read_csv(playlist_file)

playlist['Air Time'] = pd.to_datetime(playlist['Air Time'], errors='coerce')
last_play = playlist.loc[playlist['Air Time'].idxmax()]
end_time = last_play['Air Time'] + timedelta(seconds = 60 * last_play['Duration'])
HTML('<p>So far, as of %s, we have seen %d tracks with %d unique titles, from %d artists.</p>' %\
    (end_time.strftime('%b %d %I:%M%p'),
     len(playlist),
     playlist.describe(include='all')['Title']['unique'], 
     playlist.describe(include='all')['Artist']['unique']
     ))

# The Whole Thing

The playlist isn't going to be big.
Sorting by year or counting plays by artist might not be all that useful.
Let's look at the whole thing.

In [3]:
HTML(playlist.to_html(index=False, columns = ['Artist', 'Title','Year']))

Artist,Title,Year
J. J. Cale,After Midnight,0
Shocking Blue,Venus,1969
Ben Folds Five,Brick,1994
Sarah McLachlan,Sweet Surrender,1997
World Party,When The Rainbow Comes,1990
Suzanne Vega,Marlene On The Wall,1985
Big Joe Turner,Shake Rattle And Roll,1992
Wilbert Harrison,Let's Work Together (Parts 1 & 2),1994
The Pretenders,Talk Of The Town,1980
The Proclaimers,I'm Gonna Be (500 Miles),1987


### Comparision with other XPN Playlists

#### 885 Best and 88 Worst

Back in 2014 there was a very similar
[885 All Time Greatest Song](https://xpn.org/music-artist/885-countdown/2014/885-countdown-2014) countdown.
There were over 300 tracks in that that didn't happen in the long XPN 2020.
Did any only get one vote?

In [4]:
def overlapping_songs(list1, list2):
    import string
    list1a = list1.copy()
    list2a = list2.copy()
    list1a['Title_lc'] = list1a['Title'].str.lower().str.translate(None, string.punctuation).str.translate(None, string.whitespace)
    list1a['Artist_lc'] = list1a['Artist'].str.lower().str.translate(None, string.punctuation).str.translate(None, string.whitespace)
    list2a['Title_lc'] = list2a['Title'].str.lower().str.translate(None, string.punctuation).str.translate(None, string.whitespace)
    list2a['Artist_lc'] = list2a['Artist'].str.lower().str.translate(None, string.punctuation).str.translate(None, string.whitespace)
    intersect = pd.merge(list1a, list2a, how='inner', on=['Title_lc', 'Artist_lc'])
    intersect.drop(intersect.columns.difference(['Title_x','Artist_x']), 1, inplace=True)
    intersect.columns = ['Artist', 'Title']
    return intersect

    
best885_file = path.join(data_dir, '885best.csv')
best885 = pd.read_csv(best885_file)
besties = overlapping_songs(playlist, best885)
besties.to_csv(path.join(data_dir, 'XPN2020_onsies_and_885Best.csv'), index=False)

s = "<p>Of the %d One Vote Wonders played so far, " + \
    "%d were in 2014's 885 Best play list."
HTML(s % (len(playlist), len(besties)))

In [5]:
HTML(besties.to_html(index=False, columns=['Artist', 'Title',]))

Artist,Title
Ray Lamontagne,Trouble
Beck,Where It's At
